# SR3 Reader

* Reads data from SR3 file (HDF).
* Return groups, wells, special and grid properties list.
* Return 2D and 3D Time vector.
* Return a 2D or 3D property.
* Save all data to a csv file.

In [38]:
import h5py
from datetime import datetime, timedelta
import re

In [39]:
with h5py.File(r'..\sr3\base_case_3a.sr3', 'r') as file:
    print("Main Groups in the HDF file:")
    for dataset in file.keys():
        print(f'  {dataset}')

    print("\nAll Groups in the HDF file:")
    def get_type(name):
        print(f'   {name:}\t{type(file[name])}')
    file.visit(get_type)

Main Groups in the HDF file:
  General
  SpatialProperties
  Tables
  TimeSeries

All Groups in the HDF file:
   General	<class 'h5py._hl.group.Group'>
   General/ComponentTable	<class 'h5py._hl.dataset.Dataset'>
   General/EventTable	<class 'h5py._hl.dataset.Dataset'>
   General/HistoryTable	<class 'h5py._hl.dataset.Dataset'>
   General/MasterTimeTable	<class 'h5py._hl.dataset.Dataset'>
   General/NameRecordTable	<class 'h5py._hl.dataset.Dataset'>
   General/TableAssociations	<class 'h5py._hl.dataset.Dataset'>
   General/UnitConversionTable	<class 'h5py._hl.dataset.Dataset'>
   General/UnitsTable	<class 'h5py._hl.dataset.Dataset'>
   SpatialProperties	<class 'h5py._hl.group.Group'>
   SpatialProperties/000000	<class 'h5py._hl.group.Group'>
   SpatialProperties/000000/BKRGCL	<class 'h5py._hl.dataset.Dataset'>
   SpatialProperties/000000/BKRGRL	<class 'h5py._hl.dataset.Dataset'>
   SpatialProperties/000000/BKROCRW	<class 'h5py._hl.dataset.Dataset'>
   SpatialProperties/000000/BKROCW	<cl

In [40]:
f = h5py.File(r'..\sr3\base_case_3a.sr3', 'r')

In [41]:
dataset = f['General/MasterTimeTable']
day_list = {number:days for (number,days) in zip(dataset['Index'], dataset['Offset in days'])}

def parse_date(date):
    date_string = str(date)
    integer_part = int(date_string.split('.')[0])
    decimal_part = float("0." + date_string.split('.')[1])

    parsed_date = datetime.strptime(str(integer_part), "%Y%m%d")
    fraction_of_day = timedelta(days=decimal_part)
    return parsed_date + fraction_of_day

dataset = f['General/MasterTimeTable']
date_list = {number:parse_date(date) for (number,date) in zip(dataset['Index'], dataset['Date'])}

In [42]:
dataset = f['General/UnitsTable']
unit_list = {number:{'internal':in_name.decode(), 'current':out_name.decode(), 'conversion':dict()} for (number,in_name,out_name) in zip(dataset['Index'],dataset['Internal Unit'],dataset['Output Unit'])}

dataset = f['General/UnitConversionTable']
for (number, name, gain, offset) in zip(dataset['Dimensionality'],dataset['Unit Name'],dataset['Gain'],dataset['Offset']):
    unit_list[number]['conversion'][name.decode()] = (1./gain, offset * (-1.))

for d in unit_list.values():
    if d['internal'] != d['current']:
        gain, offset = d['conversion'][d['internal']]
        for k in d['conversion']:
            g, o = d['conversion'][k]
            d['conversion'][k] = (g / gain, o - offset)

def set_current_unit(dimensionality, unit):
    if unit not in unit_list[dimensionality]['conversion']:
        raise ValueError(f'{unit} is not a valid unit for the current dimension.')
    unit_list[dimensionality]['current'] = unit

def unit_conversion(value, dimensionality, unit=None):
    if unit is None:
        unit = unit_list[dimensionality]['current']
    if unit == unit_list[dimensionality]['internal']:
        return value
    gain, offset = unit_list[dimensionality]['conversion'][unit]
    if gain == 1. and offset == 0.:
        return value
    else:
        return value * gain + offset

In [43]:
dataset = f['General/ComponentTable']
component_list = {(number+1):name[0].decode() for (number,name) in enumerate(dataset[:])}

def replace_components_property_list(property_list):
    pattern = re.compile(r'\((\d+)\)')
    def replace(match):
        number = int(match.group(1))
        return f'({str(component_list.get(number, match.group(1)))})'
    return {pattern.sub(replace, k):v for k,v in property_list.items()}

In [44]:
dataset = f['General/NameRecordTable']
property_list = dict()
for (keyword,name,long_name,dimensionality) in zip(dataset['Keyword'],dataset['Name'],dataset['Long Name'],dataset['Dimensionality']):
    if keyword != '':
        keyword = keyword.decode()
        name = name.decode()
        long_name = long_name.decode()
        dimensionality = dimensionality.decode()
        if keyword[-2:] == '$C':
            for c in component_list.values():
                property_list[f'{keyword[:-2]}({c})'] = {'name':name.replace('$C', f' ({c})'), 'long name':long_name.replace('$C', f' ({c})'), 'dimensionality_string':dimensionality}
        else:
            property_list[keyword] = {'name':name, 'long name':long_name, 'dimensionality_string':dimensionality}


# Property Formulas
* Basic
    * 1035: sum (2 parameters)
    * 1037: sum (3 parameters)

* Time derivatives
    * 1040: derivative (1 parameter)
    * 1045: monthly derivative (1 parameter)
    * 1046: quarterly derivative (1 parameter)
    * 1047: yearly derivative (1 parameter)
    * 1048: daily derivative (1 parameter)
    * 1049: weekly derivative (1 parameter)

* Percentage
    * 1055: A*100 (1 parameter)

* Division
    * 1060: division (2 parameters)
    * 1062: division * 100 (2 parameters) <= used with on-fraction

    * 1080: division (2 parameters)

* Previous month
    * 1121: monthly derivative of previous month (1 parameter)
    * 1122: yearly derivative of previous month (1 parameter)

* Grid properties(?)
    * 1130: A*grid height (1 parameter)
    * 1140: A/(B*C) ? (3 parameters)
    * 1160: A*B (2 parameters)
    * 1200: ternary (2 parameters)
    * 1210: sum over layers (1 parameter)

* Per sector
    * 1110: ??? (1 parameter) <= per sector
    * 1245: ??? (1 parameter) <= per sector
    * 1246: derivative of 1245 (1 parameter) 



In [45]:
def unit_from_dimensionality(dimensionality_string):
    if dimensionality_string == '':
        return ''
    unit = ''
    if dimensionality_string[0] == '-':
        unit = '1'
    d = ''
    for c in dimensionality_string:
        if c == '|':
            unit = unit + unit_list[int(d)]['current']
            d = ''
        elif c == '-':
            unit = unit + '/'
        else:
            d = d + c
    return unit

def unit_from_property(property_):
    return unit_from_dimensionality(property_list[property_]['dimensionality_string'])

In [46]:
dataset = f['TimeSeries/WELLS/Origins']
well_list = {name.decode():number for (number,name) in enumerate(dataset[:]) if name.decode()!=''}

dataset = f['TimeSeries/WELLS/WellTable']
well_parent = {name.decode():parent.decode() for (name,parent) in zip(dataset['Name'], dataset['Parent'])}

dataset = f['TimeSeries/WELLS/Variables']
well_property_list = {name.decode():number for (number,name) in enumerate(dataset[:])}
well_property_list = replace_components_property_list(well_property_list)

dataset = f['TimeSeries/WELLS/Timesteps']
well_timestep_list = {number:timestep for (number,timestep) in enumerate(dataset[:])}

well_day_list = [day_list[ts] for ts in well_timestep_list.values()]
well_date_list = [date_list[ts] for ts in well_timestep_list.values()]

def get_well_property(property_name, well_name):
    dataset = f['TimeSeries/WELLS/Data']
    return dataset[:,well_property_list[property_name],well_list[well_name]]

In [48]:
dataset = f['TimeSeries/GROUPS/Origins']
group_list = {name.decode():number for (number,name) in enumerate(dataset[:]) if name.decode()!=''}

dataset = f['TimeSeries/GROUPS/GroupTable']
group_parent = {name.decode():parent.decode() for (name,parent) in zip(dataset['Name'], dataset['Parent'])}

dataset = f['TimeSeries/GROUPS/Variables']
group_property_list = {name.decode():number for (number,name) in enumerate(dataset[:])}
group_property_list = replace_components_property_list(group_property_list)

dataset = f['TimeSeries/GROUPS/Timesteps']
group_timestep_list = {number:timestep for (number,timestep) in enumerate(dataset[:])}

group_day_list = [day_list[ts] for ts in group_timestep_list.values()]
group_date_list = [date_list[ts] for ts in group_timestep_list.values()]

def get_group_property(property_name, group_name):
    dataset = f['TimeSeries/GROUPS/Data']
    return dataset[:,group_property_list[property_name],group_list[group_name]]

In [49]:
dataset = f['TimeSeries/LAYERS/Origins']
layer_list = {name.decode():number for (number,name) in enumerate(dataset[:]) if name.decode()!=''}

dataset = f['TimeSeries/LAYERS/LayerTable']
layer_parent = {f'{parent.decode()}{{{name.decode()}}}':parent.decode() for (name,parent) in zip(dataset['Name'], dataset['Parent'])}

dataset = f['TimeSeries/LAYERS/LayerTable']
layer_connection = {f'{parent.decode()}{{{name.decode()}}}':connection for (name,parent,connection) in zip(dataset['Name'], dataset['Parent'], dataset['Connect To'])}

dataset = f['TimeSeries/LAYERS/Variables']
layer_property_list = {name.decode():number for (number,name) in enumerate(dataset[:])}
layer_property_list = replace_components_property_list(layer_property_list)

dataset = f['TimeSeries/LAYERS/Timesteps']
layer_timestep_list = {number:timestep for (number,timestep) in enumerate(dataset[:])}

layer_day_list = [day_list[ts] for ts in layer_timestep_list.values()]
layer_date_list = [date_list[ts] for ts in layer_timestep_list.values()]

def get_layer_property(property_name, layer_name):
    dataset = f['TimeSeries/LAYERS/Data']
    return dataset[:,layer_property_list[property_name],layer_list[layer_name]]

In [50]:
dataset = f['TimeSeries/SECTORS/Origins']
sector_list = {name.decode():number for (number,name) in enumerate(dataset[:]) if name.decode()!=''}

dataset = f['TimeSeries/SECTORS/Variables']
sector_property_list = {name.decode():number for (number,name) in enumerate(dataset[:])}
sector_property_list = replace_components_property_list(sector_property_list)

dataset = f['TimeSeries/SECTORS/Timesteps']
sector_timestep_list = {number:timestep for (number,timestep) in enumerate(dataset[:])}

sector_day_list = [day_list[ts] for ts in sector_timestep_list.values()]
sector_date_list = [date_list[ts] for ts in sector_timestep_list.values()]

def get_sector_property(property_name, sector_name):
    dataset = f['TimeSeries/SECTORS/Data']
    return dataset[:,sector_property_list[property_name],sector_list[sector_name]]

In [51]:
dataset = f['TimeSeries/SPECIAL HISTORY/Variables']
special_property_list = {name.decode():number for (number,name) in enumerate(dataset[:])}

dataset = f['TimeSeries/SPECIAL HISTORY/Timesteps']
special_timestep_list = {number:timestep for (number,timestep) in enumerate(dataset[:])}
special_property_list = replace_components_property_list(special_property_list)

special_day_list = [day_list[ts] for ts in special_timestep_list.values()]
special_date_list = [date_list[ts] for ts in special_timestep_list.values()]

def get_special_property(property_name):
    dataset = f['TimeSeries/SPECIAL HISTORY/Data']
    return dataset[:,special_property_list[property_name],0]

## Grid Properties

In [67]:
dataset = f['SpatialProperties']
grid_timestep_list = dict()
i = 0
for key in dataset.keys():
    sub_dataset = f[f"SpatialProperties/{key}"]
    if isinstance(sub_dataset, h5py._hl.group.Group):
        grid_timestep_list[i] = int(key)
        i += 1
    
grid_day_list = [day_list[ts] for ts in grid_timestep_list.values()]
grid_date_list = [date_list[ts] for ts in grid_timestep_list.values()]

In [53]:
dataset = f['SpatialProperties/000000/GRID']
ni = dataset['IGNTID'][0]
nj = dataset['IGNTJD'][0]
nk = dataset['IGNTKD'][0]

In [54]:
dataset = f['SpatialProperties/Statistics']
grid_property_list = {name.decode():(min_,max_) for name,min_,max_ in zip(dataset['Keyword'],dataset['Min'],dataset['Max'])}

In [55]:
dataset = f['SpatialProperties/000000']
for key in dataset.keys():
    sub_dataset = f[f"SpatialProperties/000000/{key}"]
    try:
        print(f'  {key}: {sub_dataset.size}')
    except:
        pass

  BKRGCL: 67241
  BKRGRL: 67241
  BKROCRW: 67241
  BKROCW: 67241
  BKROGCG: 67241
  BKROGCRG: 67241
  BKRWIRO: 67241
  BKRWRO: 67241
  BSGCON: 67241
  BSGCRIT: 67241
  BSLCON: 67241
  BSOIRG: 67241
  BSOIRW: 67241
  BSORG: 67241
  BSORW: 67241
  BSWCON: 67241
  BSWCRIT: 67241
  KRSETN: 67241
  MODBVOL: 67241
  NET%2FGROSS: 67241
  PCGMAX: 67241
  PCWMAX: 67241
  PCWSHF: 67241
  PERMI: 67241
  PERMJ: 67241
  PERMK: 67241
  POR: 67241
  PRES: 67241
  SG: 67241
  SO: 67241
  SW: 67241
  TRLI: 67241
  TRLJ: 67241
  TRLK: 67241
  TRMI: 67241
  TRMJ: 67241
  TRMK: 67241
  VISG: 67241
  VISO: 67241
  VISW: 67241
  Z(1): 67241


In [59]:
dataset = f['SpatialProperties/000000/GRID']
for key in dataset.keys():
    sub_dataset = f[f"SpatialProperties/000000/GRID/{key}"]
    print(f'  {key}: {sub_dataset.size}')

  BLOCKDEPTH: 533403
  BLOCKPVOL: 67241
  BLOCKS: 4267224
  BLOCKSIZE: 1600209
  BVOL: 67241
  ICNTDR: 176572
  ICSTBC: 533403
  ICSTCG: 533403
  ICSTGN: 533403
  ICSTPB: 533403
  ICSTPS: 533403
  ICTPS1: 176572
  ICTPS2: 176572
  IGNTFR: 1
  IGNTGT: 1
  IGNTID: 1
  IGNTJD: 1
  IGNTKD: 1
  IGNTNC: 2
  IGNTNS: 2
  IGNTZA: 1
  IPSTAC: 67241
  IPSTBT: 67241
  IPSTCS: 67241
  ISECTGEOM: 67242
  NODES: 1886853
  WELLRADIUS: 1


In [60]:
grid_property_list

{'ICSTBC': (-2.0, 0.0),
 'ICSTCG': (0.0, 0.0),
 'ICSTGN': (1.0, 1.0),
 'ICSTPS': (0.0, 67241.0),
 'ICSTPB': (1.0, 533403.0),
 'IPSTCS': (373.0, 533075.0),
 'IPSTBT': (9.0, 9.0),
 'IPSTAC': (1.0, 1.0),
 'IGNTID': (47.0, 47.0),
 'IGNTJD': (39.0, 39.0),
 'IGNTKD': (291.0, 291.0),
 'IGNTZA': (0.0, 0.0),
 'IGNTGT': (12.0, 12.0),
 'IGNTFR': (533404.0, 533404.0),
 'IGNTNC': (0.0, 533403.0),
 'IGNTNS': (0.0, 533403.0),
 'ICTPS1': (1.0, 67238.0),
 'ICTPS2': (3.0, 67240.0),
 'ICNTDR': (1.0, 3.0),
 'BLOCKSIZE': (0.0, 277.3068),
 'BLOCKDEPTH': (5092.376, 5818.083),
 'NODES': (5081.534, 7282220.5),
 'BLOCKS': (1.0, 628951.0),
 'ISECTGEOM': (-1.0, 533075.0),
 'BLOCKPVOL': (2000.1254, 92200.94),
 'BVOL': (9696.793, 241620.36),
 'WELLRADIUS': (-100000000.0, -100000000.0),
 'PERMI': (1e-19, 9.0),
 'PERMJ': (1e-19, 9.0),
 'PERMK': (1e-19, 0.9),
 'TRMI': (1.0, 1.0),
 'TRMJ': (1.0, 1.0),
 'TRMK': (1.0, 1.0),
 'TRLI': (1.0, 1.0),
 'TRLJ': (1.0, 1.0),
 'TRLK': (1.0, 1.0),
 'BSWCRIT': (0.18, 0.18),
 'BSGCRIT